# Random Encryption on AES keys and generating the reports

In [17]:
import os
import csv
import base64
import secrets
import time

def generate_random_key():
    return secrets.token_hex(16)

def random_encryption(file_data):
    random_key = generate_random_key()
    R1, R2 = random_key[:len(random_key)//2], random_key[len(random_key)//2:]
    CTDevId_Uid = f"Encrypted({file_data})"
    return R1, R2, CTDevId_Uid

def concatenating_random_number(R1, CTDevId_Uid, R2):
    return f"{R1}||{CTDevId_Uid}||{R2}"

def encrypt_file(file_path, encrypted_folder, encryption_report):
    start_time = time.time()

    with open(file_path, 'rb') as file:
        file_data = file.read()

    file_data_b64 = base64.b64encode(file_data).decode('utf-8')

    R1, R2, CTDevId_Uid = random_encryption(file_data_b64)
    CT_prime = concatenating_random_number(R1, CTDevId_Uid, R2)

    encrypted_file_path = os.path.join(encrypted_folder, f"encrypted_{os.path.basename(file_path)}")
    with open(encrypted_file_path, 'w') as file:
        encoded_data = base64.b64encode(CT_prime.encode('utf-8')).decode('utf-8')
        file.write(encoded_data)

    end_time = time.time()
    encryption_duration = end_time - start_time

    # Extract user_id from the filename
    user_id_parts = os.path.basename(file_path).split('_')
    user_id = user_id_parts[2] if len(user_id_parts) > 2 else 'Unknown'

    encryption_report.append({'filename': os.path.basename(file_path), 'user_id': user_id, 'encryption_duration': encryption_duration})

def write_encryption_report(encryption_report, report_path):
    report_file_path = os.path.join(report_path, "RandomEncReport150kb.csv")
    with open(report_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['filename', 'user_id', 'encryption_duration'])
        writer.writeheader()
        for entry in encryption_report:
            writer.writerow(entry)

def encrypt_all_files_in_folder(source_folder, encrypted_folder, report_folder):
    if not os.path.exists(encrypted_folder):
        os.makedirs(encrypted_folder)
    
    encryption_report = []

    for filename in os.listdir(source_folder):
        file_path = os.path.join(source_folder, filename)
        if os.path.isfile(file_path):
            encrypt_file(file_path, encrypted_folder, encryption_report)

    if not os.path.exists(report_folder):
        os.makedirs(report_folder)
    write_encryption_report(encryption_report, report_folder)

def main():
    source_folder = input("Enter the path to the folder containing files to encrypt: ")
    encrypted_folder = input("Enter the path to the folder where encrypted files will be saved: ")
    report_folder = input("Enter the path to the folder where the encryption report will be saved: ")

    encrypt_all_files_in_folder(source_folder, encrypted_folder, report_folder)
    print("Encryption process completed. Report generated.")

if __name__ == "__main__":
    main()


Enter the path to the folder containing files to encrypt: C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/T1/850KB_AES_Enc_key
Enter the path to the folder where encrypted files will be saved: C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/T1/random[2]
Enter the path to the folder where the encryption report will be saved: C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb
Encryption process completed. Report generated.


In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/T1/850KB_AES_Enc_key

In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/T1/random[1]

# RANDOM decryption and report

In [18]:
import os
import csv
import json
import base64
import time

def random_decryption(CT_prime):
    R1, encrypted_data_b64, R2 = CT_prime.split('||')
    encrypted_data = base64.b64decode(encrypted_data_b64)  # Decode from base64 as binary
    json_data = encrypted_data[len("Encrypted(".encode()): -1]  # Remove 'Encrypted(' and the closing parenthesis
    return json_data

def decrypt_file(file_path, decrypted_folder):
    start_time = time.time()

    with open(file_path, 'rb') as file:
        encoded_data = file.read()
        CT_prime = base64.b64decode(encoded_data).decode('utf-8')

    decrypted_data = random_decryption(CT_prime)

    # Construct the decrypted filename
    original_filename = os.path.basename(file_path)
    decrypted_filename = original_filename.replace('encrypted_', '')  # Remove 'encrypted_' from the filename
    decrypted_file_path = os.path.join(decrypted_folder, decrypted_filename)

    with open(decrypted_file_path, 'wb') as file:
        file.write(decrypted_data)

    end_time = time.time()
    decryption_duration = end_time - start_time

    # Extract user_id from the filename
    user_id_parts = original_filename.split('_')
    user_id_index = user_id_parts.index('user') + 1 if 'user' in user_id_parts else -1
    user_id = user_id_parts[user_id_index] if user_id_index != -1 and user_id_index < len(user_id_parts) else 'Unknown'

    return {'filename': original_filename, 'user_id': user_id, 'decryption_duration': decryption_duration}

def write_decryption_report(decryption_report, report_path):
    report_file_path = os.path.join(report_path, "RandomDecReport.csv")
    with open(report_file_path, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=['filename', 'user_id', 'decryption_duration'])
        writer.writeheader()
        for entry in decryption_report:
            writer.writerow(entry)

def decrypt_all_files_in_folder(encrypted_folder, decrypted_folder, report_folder):
    if not os.path.exists(decrypted_folder):
        os.makedirs(decrypted_folder)
    
    decryption_report = []

    for filename in os.listdir(encrypted_folder):
        if filename.startswith("encrypted_"):
            file_path = os.path.join(encrypted_folder, filename)
            report_entry = decrypt_file(file_path, decrypted_folder)
            decryption_report.append(report_entry)

    if not os.path.exists(report_folder):
        os.makedirs(report_folder)
    write_decryption_report(decryption_report, report_folder)

def main():
    encrypted_folder = input("Enter the path to the folder containing encrypted files: ")
    decrypted_folder = input("Enter the path to the folder where decrypted files will be saved: ")
    report_folder = input("Enter the path to the folder where the decryption report will be saved: ")

    decrypt_all_files_in_folder(encrypted_folder, decrypted_folder, report_folder)
    print("Decryption process completed. Report generated.")

if __name__ == "__main__":
    main()


Enter the path to the folder containing encrypted files: C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/T1/random[2]
Enter the path to the folder where decrypted files will be saved: C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/T1/randomDec[2]
Enter the path to the folder where the decryption report will be saved: C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb
Decryption process completed. Report generated.


In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/T1/random[1]

In [ ]:
C:/Users/ROHAN/IotSimulation/OneHour/50KB/850kb/T1/randomDec[1]